In [1]:
%run -i startup.py

In [2]:
%run -i ./functions/dc_transform.py

In [3]:
%run -i ./functions/dc_join.py

data_collections = ['^cursos*','estruturas_curriculares','curriculo_componente','componentes_curriculares','turmas','matriculas_em_componentes']

for dc in data_collections:
    dc_join(db=db,dc=dc)

In [4]:
import re
for dcb in dc_join.dict.keys():
    print('%-35s' % dcb,end='') # %-35s : varchar 35 alinhado à esquerda
    c = []
    for k in dc_join.dict[dcb]['data'].keys():
        if re.findall('^id_.*',k):
            c.append(k)

    print(c,end='')
    print('')

ufrn_wait/cursos                   ['id_curso', 'id_coordenador', 'id_unidade_responsavel']
ufrn_wait/estruturas_curriculares  ['id_curriculo', 'id_curso']
ufrn_wait/curriculo_componente     ['id_curriculo_componente', 'id_curriculo', 'id_componente_curricular']
ufrn_wait/componentes_curriculares ['id_componente']
ufrn_wait/turmas                   ['id_turma', 'id_componente_curricular', 'id_turma_agrupadora']
ufrn_wait/matriculas_em_componentes['id_turma', 'id_curso']


In [5]:
cursos                   = dc_join.dict['ufrn_wait/cursos']['data']
estruturas_curriculares  = dc_join.dict['ufrn_wait/estruturas_curriculares']['data']
curriculo_componente     = dc_join.dict['ufrn_wait/curriculo_componente']['data']
componentes_curriculares = dc_join.dict['ufrn_wait/componentes_curriculares']['data']
turmas                   = dc_join.dict['ufrn_wait/turmas']['data']
matriculas_componentes   = dc_join.dict['ufrn_wait/matriculas_em_componentes']['data']

### Análise de notas por estrutura curricular

In [234]:
getin = lambda df,i,v: df[df[i].isin(v.unique())]

ect        = cursos[cursos.nome.isin(['CIÊNCIAS E TECNOLOGIA'])]

ect_ec     = getin(estruturas_curriculares, 'id_curso',ect.id_curso)
ect_cc     = getin(curriculo_componente,'id_curriculo',ect_ec.id_curriculo)
ect_ccc    = getin(componentes_curriculares,'id_componente',ect_cc.id_componente_curricular)
ect_turmas = getin(turmas,'id_componente_curricular',ect_ccc.id_componente)
ect_mc     = getin(matriculas_componentes,'id_turma',ect_turmas.id_turma)

In [266]:
ect_mc

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
52922,57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,1.0,NaN,False,0.0,NaN,0.0,TRANCADO
52923,57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,2.0,NaN,False,0.0,NaN,0.0,TRANCADO
52924,57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,3.0,NaN,False,0.0,NaN,0.0,TRANCADO
52925,57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,1.0,"3,0",False,0.0,"3,3",12.0,REPROVADO
52926,57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,2.0,"5,0",False,0.0,"3,3",12.0,REPROVADO
...,...,...,...,...,...,...,...,...,...,...
14095030,57587456,1cfe7916287771785cec4c2382cee3ed,10320810.0,2.0,"6,8",False,0.0,"7,7",6.0,APROVADO
14095031,57587456,1cfe7916287771785cec4c2382cee3ed,10320810.0,3.0,"8,0",False,0.0,"7,7",6.0,APROVADO
14095032,57587456,ee54aae6b631b10869ae46accc75fcbc,10320810.0,1.0,"9,7",False,0.0,"8,7",6.0,APROVADO
14095033,57587456,ee54aae6b631b10869ae46accc75fcbc,10320810.0,2.0,"9,5",False,0.0,"8,7",6.0,APROVADO


In [264]:
%matplotlib widget
import seaborn as sns
from matplotlib import pyplot as pt

sns.set()
pt.close(1)
fig,ax = pt.subplots(1,1,figsize=(10,3),num=1)
sns.ecdfplot(ect_mc.discente.value_counts().values)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:ylabel='Proportion'>

In [232]:
x

AttributeError: 'DataFrame' object has no attribute 'summary'

In [ ]:
ect_cc.id_componente_curricular

177      57601
257      57599
381      57590
398      57628
502      57628
         ...  
78538    41437
78597    57606
78611    57587
78701    57606
78718    57607
Name: id_componente_curricular, Length: 1149, dtype: int64

In [204]:
ect_cc.id_componente_curricular.unique().shape

(306,)

In [203]:
ect_ccs = 

177      57601
257      57599
381      57590
398      57628
502      57628
         ...  
78538    41437
78597    57606
78611    57587
78701    57606
78718    57607
Name: id_componente_curricular, Length: 1149, dtype: int64

In [189]:
estruturas_curriculares.shape

(397, 28)

In [187]:
curriculo_componente.shape

(94509, 6)

,id_curriculo_componente,id_curriculo,id_componente_curricular,semestre_oferta,tipo_vinculo_componente,nivel_ensino
3,114814991,102187364,59886,0,OPTATIVO,GRADUAÇÃO
12,127681085,127681072,65892,0,OPTATIVO,GRADUAÇÃO
13,133814114,133814073,24849,1,OBRIGATÓRIO,GRADUAÇÃO
21,137233,100668,28203,3,OBRIGATÓRIO,GRADUAÇÃO
27,140112786,140112779,24882,0,OPTATIVO,GRADUAÇÃO
...,...,...,...,...,...,...
94502,124868749,124868739,59280,0,OPTATIVO,MESTRADO
94503,116368020,116367936,61627,0,OPTATIVO,DOUTORADO
94505,129993722,129993616,40499,0,OPTATIVO,DOUTORADO
94506,123438362,123438351,42272,0,OBRIGATÓRIO,MESTRADO


In [180]:
estruturas_curriculares.iloc[0]

id_curriculo                                                    95617859
codigo                                                                01
nome_matriz                     GESTÃO PÚBLICA - CAICÓ - I - TECNOLÓGICO
id_curso                                                        92017023
nome_curso                                                GESTÃO PÚBLICA
semestre_conclusao_minimo                                              5
semestre_conclusao_ideal                                               5
semestre_conclusao_maximo                                              6
meses_conclusao_minimo                                               NaN
meses_conclusao_ideal                                                NaN
meses_conclusao_maximo                                               NaN
cr_total_minimo                                                      114
ch_total_minima                                                     1710
ch_optativas_minima                                

In [178]:
ect_ec.id_curriculo

,id_curriculo_componente,id_curriculo,id_componente_curricular,semestre_oferta,tipo_vinculo_componente,nivel_ensino
0,89767482,27289611,43042,4,OBRIGATÓRIO,GRADUAÇÃO
1,127083,100771,27659,0,OPTATIVO,GRADUAÇÃO
2,124982112,51401944,48472,0,OPTATIVO,GRADUAÇÃO
3,114814991,102187364,59886,0,OPTATIVO,GRADUAÇÃO
4,108707,100490,21159,3,OBRIGATÓRIO,GRADUAÇÃO
...,...,...,...,...,...,...
94504,140800,140066,37107,1,OBRIGATÓRIO,DOUTORADO
94505,129993722,129993616,40499,0,OPTATIVO,DOUTORADO
94506,123438362,123438351,42272,0,OBRIGATÓRIO,MESTRADO
94507,51415200,51411146,44415,0,OPTATIVO,MESTRADO


In [172]:
ect_cc = getin(curriculo_componente, 'id_curriculo',ect_ec.id_curriculo)

In [173]:
ect_cc

,id_curriculo_componente,id_curriculo,id_componente_curricular,semestre_oferta,tipo_vinculo_componente,nivel_ensino


In [163]:
ect_ec = pd.DataFrame([ec for ec in estruturas_curriculares.iloc if 'CIÊNCIAS E TECNOLOGIA' in ec.nome_curso])

In [7]:
ect_curso = cursos[cursos['id_curso'].isin(ect_ec.id_curso)]

In [8]:
ect_componentes = curriculo_componente[curriculo_componente.id_curriculo.isin(ect_ec.id_curriculo)]

In [9]:
ect_componente_curriculo = ect_componentes.groupby('id_curriculo')

In [10]:
for c,curriculo_id in enumerate(ect_ec.id_curriculo.iloc):
    ect_curriculo = ect_componente_curriculo.get_group(curriculo_id)
    this_turmas = turmas.id_componente_curricular.isin(ect_curriculo.id_componente_curricular)
    if c == 0:
        ect_turmas_id = this_turmas
    else:
        ect_turmas_id = ect_turmas_id | this_turmas

In [11]:
ect_turmas = turmas[ect_turmas_id]
ect_turmas = ect_turmas.set_index('id_turma')

In [12]:
ect_turmas

,codigo_turma,siape,matricula_docente_externo,observacao,id_componente_curricular,ch_dedicada_periodo,nivel_ensino,campus_turma,local,ano,...,capacidade_aluno,tipo,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes
id_turma,,,,,,,,,,,,,,,,,,,,,
1076015,01,1149549.0,NaN,NaN,24772,60,GRADUAÇÃO,NaN,NaN,2000,...,30.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
1076016,02,252325.0,NaN,NaN,24772,60,GRADUAÇÃO,NaN,NaN,2000,...,45.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
1076017,03,252300.0,NaN,NaN,24772,60,GRADUAÇÃO,NaN,NaN,2000,...,49.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
1076920,01,346923.0,NaN,NaN,24856,60,GRADUAÇÃO,NaN,I B 2,2000,...,60.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
1076921,02,346923.0,NaN,NaN,24856,60,GRADUAÇÃO,NaN,V F 5,2000,...,50.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117933,1,1451117.0,NaN,NaN,27095,60,GRADUAÇÃO,NaN,I-C1,2005,...,60.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
1118068,1,1440116.0,NaN,NaN,24856,60,GRADUAÇÃO,NaN,V F 2,2005,...,65.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial
1118343,1,252325.0,NaN,NaN,24772,60,GRADUAÇÃO,NaN,4E5,2005,...,50.0,REGULAR,False,NaN,False,NaN,NaN,CONSOLIDADA,NaN,Presencial


In [13]:
ect_matriculas = matriculas_componentes[matriculas_componentes['id_turma'].isin(ect_turmas.index.unique())]
ect_matriculas = ect_matriculas.set_index('id_turma')

In [27]:
ect_matriculas.head(5)

,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
id_turma,,,,,,,,,
57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,1.0,NaN,False,0.0,NaN,0.0,TRANCADO
57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,2.0,NaN,False,0.0,NaN,0.0,TRANCADO
57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,3.0,NaN,False,0.0,NaN,0.0,TRANCADO
57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,1.0,"3,0",False,0.0,"3,3",12.0,REPROVADO
57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,2.0,"5,0",False,0.0,"3,3",12.0,REPROVADO


In [ ]:
ect_matriculas

In [146]:
discentes_ect_cursos_count = ect_matriculas.id_curso.value_counts()

In [153]:
getin(ect_matriculas,'id_curso',*cursos.id_curso.unique())

,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
id_turma,,,,,,,,,
57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,1.0,NaN,False,0.0,NaN,0.0,TRANCADO
57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,2.0,NaN,False,0.0,NaN,0.0,TRANCADO
57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,3.0,NaN,False,0.0,NaN,0.0,TRANCADO
57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,1.0,"3,0",False,0.0,"3,3",12.0,REPROVADO
57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,2.0,"5,0",False,0.0,"3,3",12.0,REPROVADO
...,...,...,...,...,...,...,...,...,...
57587456,1cfe7916287771785cec4c2382cee3ed,10320810.0,2.0,"6,8",False,0.0,"7,7",6.0,APROVADO
57587456,1cfe7916287771785cec4c2382cee3ed,10320810.0,3.0,"8,0",False,0.0,"7,7",6.0,APROVADO
57587456,ee54aae6b631b10869ae46accc75fcbc,10320810.0,1.0,"9,7",False,0.0,"8,7",6.0,APROVADO


In [95]:
curso_id

111635063.0

In [93]:
for curso_id,curso_id_count in vc_zip(discentes_ect_cursos_count):
    curso_nome = cursos[cursos.id_curso.isin([curso_id])].nome
    curso_nome = curso_nome.iloc[0]
    print(curso_id_count,':',curso_nome,end='\n')
    

564129 : CIÊNCIAS E TECNOLOGIA
21246 : ENGENHARIA DE COMPUTAÇÃO
13349 : ENGENHARIA ELÉTRICA
8697 : ENGENHARIA AMBIENTAL
6641 : ENGENHARIA MECÂNICA
5190 : ENGENHARIA DE MATERIAIS
4219 : ENGENHARIA MECATRÔNICA
3988 : ENGENHARIA DE PETRÓLEO
3501 : ENGENHARIA BIOMÉDICA
3354 : CIÊNCIAS BIOLÓGICAS
3264 : ENGENHARIA CIVIL
2018 : HISTÓRIA
1984 : TECNOLOGIA DA INFORMAÇÃO
1839 : ADMINISTRAÇÃO
1836 : QUÍMICA
1790 : CIÊNCIA DA COMPUTAÇÃO
1725 : FÍSICA
1682 : ENGENHARIA DE TELECOMUNICAÇÕES
1631 : MATEMÁTICA
1620 : DIREITO
1358 : MÚSICA
1321 : FARMÁCIA
1302 : EDUCAÇÃO FÍSICA
1272 : COMUNICAÇÃO SOCIAL
1263 : CIÊNCIAS BIOLÓGICAS
1245 : CIÊNCIAS ECONÔMICAS


IndexError: single positional indexer is out-of-bounds

In [14]:
print(len(ect_turmas.index),len(ect_turmas.index.unique()))

7848 6425


In [274]:
ect_matriculas

id_curso  unidade nota reposicao  \
id_turma discente                                                               
57509567 6fe71d0fc29c75d16344137fb7a2e831   2000026.0      1.0  NaN     False   
         6fe71d0fc29c75d16344137fb7a2e831   2000026.0      2.0  NaN     False   
         6fe71d0fc29c75d16344137fb7a2e831   2000026.0      3.0  NaN     False   
         699c448537aa16f077e18b2c7f81e0ae   2000026.0      1.0  3,0     False   
         699c448537aa16f077e18b2c7f81e0ae   2000026.0      2.0  5,0     False   
...                                               ...      ...  ...       ...   
57587456 1cfe7916287771785cec4c2382cee3ed  10320810.0      2.0  6,8     False   
         1cfe7916287771785cec4c2382cee3ed  10320810.0      3.0  8,0     False   
         ee54aae6b631b10869ae46accc75fcbc  10320810.0      1.0  9,7     False   
         ee54aae6b631b10869ae46accc75fcbc  10320810.0      2.0  9,5     False   
         ee54aae6b631b10869ae46accc75fcbc  10320810.0      3.0  7,0     False   

                                           faltas_unidade media_final  \
id_turma discente                                                       
57509567 6fe71d0fc29c75d16344137fb7a2e831             0.0         NaN   
         6fe71d0fc29c75d16344137fb7a2e831             0.0         NaN   
         6fe71d0fc29c75d16344137fb7a2e831             0.0         NaN   
         699c448537aa16f077e18b2c7f81e0ae             0.0         3,3   
         699c448537aa16f077e18b2c7f81e0ae             0.0         3,3   
...                                                   ...         ...   
57587456 1cfe7916287771785cec4c2382cee3ed             0.0         7,7   
         1cfe7916287771785cec4c2382cee3ed             0.0         7,7   
         ee54aae6b631b10869ae46accc75fcbc             0.0         8,7   
         ee54aae6b631b10869ae46accc75fcbc             0.0         8,7   
         ee54aae6b631b10869ae46accc75fcbc             0.0         8,7   

                                           numero_total_faltas  descricao  
id_turma discente                                                          
57509567 6fe71d0fc29c75d16344137fb7a2e831                  0.0   TRANCADO  
         6fe71d0fc29c75d16344137fb7a2e831                  0.0   TRANCADO  
         6fe71d0fc29c75d16344137fb7a2e831                  0.0   TRANCADO  
         699c448537aa16f077e18b2c7f81e0ae                 12.0  REPROVADO  
         699c448537aa16f077e18b2c7f81e0ae                 12.0  REPROVADO  
...                                                        ...        ...  
57587456 1cfe7916287771785cec4c2382cee3ed                  6.0   APROVADO  
         1cfe7916287771785cec4c2382cee3ed                  6.0   APROVADO  
         ee54aae6b631b10869ae46accc75fcbc                  6.0   APROVADO  
         ee54aae6b631b10869ae46accc75fcbc                  6.0   APROVADO  
         ee54aae6b631b10869ae46accc75fcbc                  6.0   APROVADO  

[693708 rows x 8 columns]

In [243]:
ect_matriculas.join(ect_turmas)

NotImplementedError: Index._join_level on non-unique index is not implemented

In [185]:
ect_turmas_ano = ect_turmas[['id_turma','ano']]

In [186]:
ect_turmas_ano

,id_turma,ano
467,1076015,2000
468,1076016,2000
469,1076017,2000
1442,1076920,2000
1443,1076921,2000
...,...,...
389682,1117933,2005
389875,1118068,2005
390183,1118343,2005
390194,1118353,2005


In [207]:
z = ect_matriculas.join(ect_turmas_ano,on=['id_turma'],rsuffix='b',how='right')
z

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,id_turmab,ano
NaN,467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1076015,2000
NaN,468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1076016,2000
NaN,469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1076017,2000
NaN,1442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1076920,2000
NaN,1443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1076921,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,389682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1117933,2005
NaN,389875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1118068,2005
NaN,390183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1118343,2005
NaN,390194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1118353,2005


In [ ]:
ect_matriculas

In [208]:
ect_matriculas.discente.unique().shape

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
52922,57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,1.0,NaN,False,0.0,NaN,0.0,TRANCADO
52923,57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,2.0,NaN,False,0.0,NaN,0.0,TRANCADO
52924,57509567,6fe71d0fc29c75d16344137fb7a2e831,2000026.0,3.0,NaN,False,0.0,NaN,0.0,TRANCADO
52925,57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,1.0,"3,0",False,0.0,"3,3",12.0,REPROVADO
52926,57509567,699c448537aa16f077e18b2c7f81e0ae,2000026.0,2.0,"5,0",False,0.0,"3,3",12.0,REPROVADO
...,...,...,...,...,...,...,...,...,...,...
14095030,57587456,1cfe7916287771785cec4c2382cee3ed,10320810.0,2.0,"6,8",False,0.0,"7,7",6.0,APROVADO
14095031,57587456,1cfe7916287771785cec4c2382cee3ed,10320810.0,3.0,"8,0",False,0.0,"7,7",6.0,APROVADO
14095032,57587456,ee54aae6b631b10869ae46accc75fcbc,10320810.0,1.0,"9,7",False,0.0,"8,7",6.0,APROVADO
14095033,57587456,ee54aae6b631b10869ae46accc75fcbc,10320810.0,2.0,"9,5",False,0.0,"8,7",6.0,APROVADO


In [176]:
ect_matriculas.discente.unique().shape

(35052,)

In [152]:
set(ect_matriculas.id_turma.unique())

{57606144,
 57638979,
 1179743,
 57573481,
 57573482,
 57573483,
 57573484,
 57573485,
 57573486,
 57573488,
 57573496,
 57573499,
 57573500,
 57573501,
 57573502,
 57573503,
 57573505,
 57573506,
 57573507,
 57573508,
 57573509,
 57573511,
 57573512,
 57573513,
 57573518,
 57573519,
 57573520,
 57573522,
 57573524,
 57573525,
 57573529,
 57573530,
 57573531,
 57573532,
 57573533,
 57573534,
 57573535,
 57573536,
 57573537,
 57573538,
 57573539,
 57573540,
 57639079,
 57573581,
 57573582,
 57573583,
 57573584,
 57573585,
 57573586,
 57573587,
 57573588,
 57573589,
 57573590,
 57573591,
 57573593,
 57573594,
 57573595,
 57573596,
 57573598,
 57573599,
 57573600,
 57573601,
 57573602,
 57573603,
 57573604,
 57573605,
 57573606,
 57573607,
 57573608,
 57573609,
 57573610,
 57573611,
 57573612,
 57639231,
 57639264,
 1180001,
 1180002,
 1212773,
 57639286,
 1212794,
 1212796,
 57606526,
 1212803,
 1212804,
 1212810,
 1212812,
 1180045,
 1180046,
 1180047,
 1180048,
 1212814,
 1212813,
 121

In [134]:
ect_matricula_turma

In [142]:
.groupby('id_turma')

1076015

In [143]:
ect_matricula_turma.get_group(ect_turmas.iloc[0].id_turma)

KeyError: 1076015

In [138]:
for turma_id in ect_turmas.id_turma:
    print(turma_id)

1076015
1076016
1076017
1076920
1076921
1077030
1099391
1082869
1082870
1083805
1083806
1083918
1084367
1086448
1086449
1087350
1087351
1087466
1087467
1090168
1090169
1090170
1091154
1091282
1091283
1091746
1093820
1093822
1094105
1094278
1094280
1095015
1096558
1096738
1100652
1101597
1101922
1101923
1102485
1103449
1103449
1103617
1104908
1104911
1105378
1105507
1105546
1105899
1105900
1109177
1109438
1109989
1110266
1110865
1111511
1112124
1112269
1113076
1113100
1115114
1115117
1120006
1120015
1122499
1123130
1123190
1123357
1123367
1124732
1125277
1126440
1126830
1127320
1127394
1130332
1130334
1130468
1130739
1130821
1130934
1133322
1134692
1134694
1134698
1134701
1134701
1134981
1135221
1135555
1135557
1135558
1135913
1136310
1136310
1140728
1140744
1140833
1141083
1141260
1141284
1141329
1141358
1141364
1141370
1141374
1141376
1141390
1141799
1141813
1141814
1141816
1141944
1141964
1141965
1144064
57228352
57228356
57228357
57228430
57228434
57228438
57228450
57228458
57228460